In [15]:
!pip install -q streamlit scikit-learn pandas numpy huggingface_hub pyngrok

In [28]:
import os

# Hugging Face token
os.environ["HF_TOKEN"]

# ngrok token
os.environ["NGROK_AUTHTOKEN"]

print("HF_TOKEN set:", "HF_TOKEN" in os.environ)
print("NGROK_AUTHTOKEN set:", "NGROK_AUTHTOKEN" in os.environ)

HF_TOKEN set: True
NGROK_AUTHTOKEN set: True


In [34]:
# ======================================================
# NIFTY AI MARKET BIAS PREDICTOR
# ======================================================
# A notebook-based ML system for estimating next-day
# directional bias using OHLC price action
# ======================================================

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from datetime import datetime
import os
from huggingface_hub import InferenceClient

print("==============================================")
print("        NIFTY AI MARKET BIAS PREDICTOR")
print("==============================================\n")

# ======================================================
# LOAD HISTORICAL DATA
# ======================================================
df = pd.read_csv("NIFTY_50 modify.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df = df.sort_values("Date").reset_index(drop=True)
df = df.drop(columns=["Adj Close"])

# ======================================================
# FEATURE ENGINEERING (HISTORICAL)
# ======================================================
df["Return_1D"] = df["Close"].pct_change()
df["Return_5D"] = df["Close"].pct_change(5)

df["SMA_20"] = df["Close"].rolling(20).mean()
df["SMA_50"] = df["Close"].rolling(50).mean()

df["Above_SMA20"] = (df["Close"] > df["SMA_20"]).astype(int)
df["Above_SMA50"] = (df["Close"] > df["SMA_50"]).astype(int)

# Target: next-day direction
df["Target"] = (df["Return_1D"].shift(-1) > 0).astype(int)

df = df.dropna().reset_index(drop=True)

features = ["Return_1D", "Return_5D", "Above_SMA20", "Above_SMA50"]
X = df[features]
y = df["Target"]

# ======================================================
# TRAIN MODEL (ONCE)
# ======================================================
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

baseline_prob = model.predict_proba(X)[:, 1].mean()
print(f"Model baseline probability of up-move: {baseline_prob:.2%}")

# ======================================================
# OPTIONAL: HUGGING FACE CHATBOT (EXPLANATION ONLY)
# ======================================================
llm = None
if "HF_TOKEN" in os.environ:
    try:
        llm = InferenceClient(
            model="meta-llama/Llama-3.1-8B-Instruct",
            token=os.environ["HF_TOKEN"]
        )
        print("Hugging Face chatbot enabled.\n")
    except Exception as e:
        print("Chatbot disabled:", e, "\n")
else:
    print("HF_TOKEN not found → chatbot disabled.\n")

# ======================================================
# USER LOOP (TRY AGAIN ENABLED)
# ======================================================
while True:
    print("\nEnter YESTERDAY'S OHLC values:")

    try:
        open_p  = float(input("Open  : "))
        high_p  = float(input("High  : "))
        low_p   = float(input("Low   : "))
        close_p = float(input("Close : "))
    except ValueError:
        print("❌ Invalid input. Please enter numeric values.")
        continue

    # ---------------- VALIDATION ----------------
    if not (low_p <= open_p <= high_p and low_p <= close_p <= high_p):
        print("❌ Invalid OHLC structure. Try again.")
        continue

    # ---------------- PRICE ACTION ----------------
    candle_body = (close_p - open_p) / open_p

    # ---------------- APPEND YESTERDAY ----------------
    df_live = df.copy()
    new_row = {
        "Date": datetime.today(),
        "Open": open_p,
        "High": high_p,
        "Low": low_p,
        "Close": close_p
    }
    df_live = pd.concat([df_live, pd.DataFrame([new_row])], ignore_index=True)

    # ---------------- RECOMPUTE FEATURES ----------------
    df_live["Return_1D"] = df_live["Close"].pct_change()
    df_live["Return_5D"] = df_live["Close"].pct_change(5)
    df_live["SMA_20"] = df_live["Close"].rolling(20).mean()
    df_live["SMA_50"] = df_live["Close"].rolling(50).mean()
    df_live["Above_SMA20"] = (df_live["Close"] > df_live["SMA_20"]).astype(int)
    df_live["Above_SMA50"] = (df_live["Close"] > df_live["SMA_50"]).astype(int)

    df_live = df_live.dropna()

    latest = df_live[features].iloc[-1:]
    prob = model.predict_proba(latest)[0][1]
    delta = prob - baseline_prob

    # ---------------- FINAL DECISION ----------------
    if candle_body < -0.005:
        signal = "BEARISH"
        reason = "Strong bearish candle override"
    elif candle_body > 0.005 and delta > 0:
        signal = "BULLISH"
        reason = "Bullish candle with positive statistical edge"
    else:
        signal = "NO TRADE"
        reason = "No clear price-action or statistical edge"

    # ---------------- OUTPUT ----------------
    print("\n================ AI MARKET OUTLOOK ================")
    print(f"Project               : NIFTY AI Market Bias Predictor")
    print(f"Prediction Date       : {datetime.today().date()}")
    print(f"Baseline Probability  : {baseline_prob:.2%}")
    print(f"Today's Probability   : {prob:.2%}")
    print(f"Candle Body (%)       : {candle_body*100:.2f}%")
    print(f"\nFINAL SIGNAL          : {signal}")
    print(f"REASON                : {reason}")
    print("===================================================")

    # ---------------- CHATBOT EXPLANATION ----------------
    if llm:
        print("\n🤖 AI Explanation:")
        try:
            explanation = llm.chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": "You are a professional Indian index trader explaining market bias clearly."
                    },
                    {
                        "role": "user",
                        "content": f"""
Yesterday OHLC:
Open={open_p}, High={high_p}, Low={low_p}, Close={close_p}

Model Output:
Signal={signal}
Probability={prob:.2%}

Explain why this bias makes sense for today's NIFTY trade.
"""
                    }
                ],
                max_tokens=200
            )
            print(explanation.choices[0].message.content)
        except Exception as e:
            print("Chatbot error:", e)

    # ---------------- TRY AGAIN ----------------
    again = input("\n🔁 Try again with new values? (y/n): ").strip().lower()
    if again != "y":
        print("\n👋 Exiting NIFTY AI Market Bias Predictor. Trade safe.")
        break

        NIFTY AI MARKET BIAS PREDICTOR

Model baseline probability of up-move: 53.08%
Hugging Face chatbot enabled.


Enter YESTERDAY'S OHLC values:


KeyboardInterrupt: Interrupted by user